# Speech Recognition


## Imports

In [14]:
import pickle
import librosa
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import matplotlib.pyplot as plt
from IPython.display import Image
import plotly.graph_objects as go

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, TensorBoard

In [15]:
sns.set()
plt.style.use('ggplot')
%matplotlib inline
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)
sys.path.append(os.path.abspath(os.path.join('../scripts')))

In [16]:
from clean_audio import CleanAudio
from file_handler import FileHandler
from audio_vis import AudioVis

In [17]:
clean_audio = CleanAudio()
file_handler = FileHandler()
audio_vis = AudioVis()

## Load Data


In [9]:
PATH_TRAIN_WAV = "../data/AMHARIC_CLEAN/train/wav/"
PATH_TEST_WAV = "../data/AMHARIC_CLEAN/test/wav/"

In [26]:
data = pd.read_csv(r'../data/final_data.csv')
data.sort_values(by=["duration"], inplace=True)
data.head(5)

,Unnamed: 0,key,text,char_length,duration,speed,category
5402,5655,tr_5302_tr54003,ሼራተን አዲስ ተመረቀ,13,2.05,6.35,Train
10003,10511,tr_9674_tr094116,ፖሊሱ እስረኞቹን ቆጠረ,14,2.05,6.84,Train
6674,7012,tr_6524_tr66025,ይህ ቀስ በቀስ እያደገ ሄደ,17,2.05,8.30,Train
8285,8686,tr_8030_tr81031,ኢነጋማ ህጋዊ እውቅና አገኘ,17,2.05,8.30,Train
6017,6314,tr_5897_tr59098,በተጨባጭ ስና የው ግን ባዶ ነው,20,2.05,9.77,Train


## Tokenizer

In [42]:
class TokenizerWrap(Tokenizer):
    def __init__(self, texts, padding, len_sent, filters, reverse=False):
        Tokenizer.__init__(self, filters=filters, char_level=True)

        self.len_sent = len_sent
        self.fit_on_texts(texts)

        self.index_to_word = dict(zip(self.word_index.values(), self.word_index.keys()))
        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            self.tokens = [list(reversed(x)) for x in self.tokens]
            truncating = 'pre'
        else:
            truncating = 'post'

        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=len_sent,
                                           padding=padding,
                                           truncating=truncating
                                           )

    def token_to_word(self, token):
        word = " " if token == 0 else self.index_to_word[token]
        return word

    def tokens_to_string(self, tokens):
        words = [self.index_to_word[token] for token in tokens if token != 0]
        text = "".join(words)
        return text

    def text_to_tokens(self, text, reverse=False, padding=False):
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:
            tokens = np.flip(tokens, axis=1)
            truncating = 'pre'
        else:
            truncating = 'post'

        if padding:
            tokens = pad_sequences(tokens,
                                   maxlen=self.len_sent,
                                   padding=truncating,
                                   truncating=truncating
                                   )
        return tokens


In [43]:
MAX_SENTENCE_LENGTH = 125       # The longest sentence in the data is around 150 chars
filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n።”፤፦’፥'  # { ።”፤፦’፥' } unique for amharic

In [44]:
%%time
tokenizer = TokenizerWrap(texts=data.text,
                          padding='post',
                          reverse=False,
                          len_sent=MAX_SENTENCE_LENGTH,
                          filters=filters)

CPU times: user 543 ms, sys: 22.6 ms, total: 566 ms
Wall time: 618 ms


In [45]:
print(len(tokenizer.word_index))
print(tokenizer.word_index)

222
{' ': 1, 'ን': 2, 'ት': 3, 'ው': 4, 'ስ': 5, 'ያ': 6, 'የ': 7, 'ተ': 8, 'በ': 9, 'አ': 10, 'ል': 11, 'እ': 12, 'ለ': 13, 'ር': 14, 'መ': 15, 'ም': 16, 'ች': 17, 'ና': 18, 'ደ': 19, 'ነ': 20, 'ገ': 21, 'ማ': 22, 'ባ': 23, 'ይ': 24, 'ሚ': 25, 'ግ': 26, 'ራ': 27, 'ቸ': 28, 'ላ': 29, 'ብ': 30, 'ድ': 31, 'ረ': 32, 'ሰ': 33, 'ከ': 34, 'ወ': 35, 'ኢ': 36, 'ታ': 37, 'ዳ': 38, 'ክ': 39, 'ዮ': 40, 'ዋ': 41, 'ህ': 42, 'ጵ': 43, 'ጥ': 44, 'ቀ': 45, 'ሪ': 46, 'ጠ': 47, 'ቅ': 48, 'ዲ': 49, 'ሳ': 50, 'ዎ': 51, 'ሮ': 52, 'ሩ': 53, 'ሉ': 54, 'ሆ': 55, 'ሁ': 56, 'ጋ': 57, 'ሊ': 58, 'ቶ': 59, 'ካ': 60, 'ፈ': 61, 'ጣ': 62, 'ፍ': 63, 'ሀ': 64, 'ሞ': 65, 'ሽ': 66, 'ዊ': 67, 'ዘ': 68, 'ቱ': 69, 'ሬ': 70, 'ኤ': 71, 'ኮ': 72, 'ሎ': 73, 'ኛ': 74, 'ዛ': 75, 'ሲ': 76, 'ቃ': 77, 'ጉ': 78, 'ቡ': 79, 'ቻ': 80, 'ዝ': 81, 'ፕ': 82, 'ቢ': 83, 'ዚ': 84, 'ኑ': 85, 'ሙ': 86, 'ሶ': 87, 'ጀ': 88, 'ቁ': 89, 'ኖ': 90, 'ኩ': 91, 'ቋ': 92, 'ሌ': 93, 'ቤ': 94, 'ሱ': 95, 'ኒ': 96, 'ቹ': 97, 'ኝ': 98, 'ጸ': 99, 'ዱ': 100, 'ቲ': 101, 'ጅ': 102, 'ሸ': 103, 'ዜ': 104, 'ቴ': 105, 'ቆ': 106, 'ዙ': 107, 'ዴ': 108, 'ኔ': 109, 'ጡ': 110, 'ኙ'

In [46]:
data.text[1]

'የጠመንጃ ተኩስ ተከፈተና አራት የኤርትራ ወታደሮች ተገደሉ'

In [47]:
sample = tokenizer.text_to_tokens(data.text[1], padding=True)
sample

array([[  7,  47,  15,   2, 118,   1,   8,  91,   5,   1,   8,  34,  61,
          8,  18,   1,  10,  27,   3,   1,   7,  71,  14,   3,  27,   1,
         35,  37,  19,  52,  17,   1,   8,  21,  19,  54,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [48]:
print(tokenizer.tokens_to_string(sample[0]))

የጠመንጃ ተኩስ ተከፈተና አራት የኤርትራ ወታደሮች ተገደሉ


save token

In [49]:
with open('../models/char_tokenizer_amharic.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle)

NameError: name 'pickle' is not defined